In [1]:
import sys
sys.path.append('../')

import json
import os
import pandas as pd
import common
from shutil import copyfile

## Filter useful images

En esta primer etapa se filtran imágenes tagueadas como útiles y que tengan ángulo asignado

In [46]:
INPUT_STATE_FILENAME = "state.json"
OUTPUT_STATE_FILENAME = "cleaned_state.json"

with open(INPUT_STATE_FILENAME) as f:
    input_state = json.load(f)

def is_useful(row):
    return row["useful"] == "yes" and not(row["photo_angle"] in [None, "other", ""])
    
useful_state = {}
for (k,v) in input_state.items():
    if (is_useful(v)):
        useful_state[k] = v
        
print("total: {}".format(len(input_state)))
print("utiles: {}".format(len(useful_state)))

total: 3681
utiles: 1639


## Filter broken part images

En esta segunda etapa se filtran aquellas imágenes que tengan alguna parte rota, en base al ángulo etiquetado

In [51]:
broken_parts = common.load_complaint_parts("../preprocessing/piezas_normalizadas.csv", False, None, ignore_syc=True)

def has_broken_part(complaint, angle):
    complaint_parts = broken_parts[broken_parts["DENUNCIA"] == complaint]["pieza_normalizada"].tolist()
    angle_parts = common.angulo_pieza[angle]
    
    return len(set(complaint_parts) & set(angle_parts)) > 0

useful_and_broken_state = {}
for (k,v) in useful_state.items():
    complaint = k.split("/")[0]
    if has_broken_part(complaint, v["photo_angle"]):
        useful_and_broken_state[k] = v

print("total: {}".format(len(input_state)))
print("utiles: {}".format(len(useful_state)))
print("utiles y con parte rota: {}".format(len(useful_and_broken_state)))

total: 3681
utiles: 1639
utiles y con parte rota: 1189


## Save new state

En esta etapa se guarda el nuevo state con el nombre de "cleaned_state.json", que debe ser copiado al image analyzer (y cambiar el nombre a state.json)

In [52]:
with open(OUTPUT_STATE_FILENAME, 'w') as f:
    json.dump(useful_and_broken_state, f)

## Adjust images

Finalmente, se ajustan las imágenes que quedaron filtradas. Se debe setear correctamente la variable IMGS_FOLDER, y dentro de ese directorio se eliminaran las imágenes que no apliquen
RECOMENDADO: hacer un backup antes.

In [53]:
import os
import re
from fnmatch import fnmatch

IMGS_FOLDER = "/Users/fcuevas/Documents/fing/tesis/image-analyzer/imgs/"

pattern = "*\.jpg"

count_to_keep = 0
to_delete = []

for path, subdirs, files in os.walk(IMGS_FOLDER):
    for subdir in subdirs:
        for inner_path, _, inner_files in os.walk(os.path.join(IMGS_FOLDER, subdir)):
            for name in inner_files:
                if name.endswith(".jpg") or name.endswith(".jpeg") or name.endswith(".png"):
                    key = subdir + "/" + name
                    if key in useful_and_broken_state:
                        count_to_keep += 1
                    else:
                        to_delete.append(os.path.join(inner_path, name))
                    
print("To keep: {}".format(count_to_keep))
print("To delete: {}".format(len(to_delete)))

To keep: 1189
To delete: 2492


In [54]:
for elem in to_delete:
    os.remove(elem)

In [40]:
common.angulo_pieza

{'frente': ['Capot',
  'Farol Delantero Izquierdo',
  'Paragolpe Delantero',
  'Farol Delantero Derecho'],
 'atras': ['Panel Trasero',
  'Farol Trasero Derecho',
  'Farol Trasero Izquierdo',
  'Paragolpe Trasero',
  'Tapa de Baul'],
 'lado_cond': ['Guardabarro Delantero Izquierdo',
  'Lateral de Caja Izquierdo',
  'Puerta Trasera Izquierda',
  'Puerta Delantera Izquierda',
  'Zocalo Izquierdo',
  'Farol Trasero Izquierdo',
  'Guardabarro Trasero Izquierdo',
  'Farol Delantero Izquierdo'],
 'lado_acomp': ['Zocalo Derecho',
  'Puerta Delantera Derecha',
  'Farol Delantero Derecho',
  'Farol Trasero Derecho',
  'Lateral de Caja Derecho',
  'Guardabarro Trasero Derecho',
  'Guardabarro Delantero Derecho',
  'Puerta Trasera Derecha'],
 'frente_cond': ['Guardabarro Delantero Izquierdo',
  'Paragolpe Delantero',
  'Lateral de Caja Izquierdo',
  'Capot',
  'Puerta Delantera Izquierda',
  'Puerta Trasera Izquierda',
  'Zocalo Izquierdo',
  'Guardabarro Trasero Izquierdo',
  'Farol Delantero Izq

In [44]:
result = {}
for (k, v) in common.angulo_pieza.items():
    newValue = []
    for elem in v:
        newValue.append(elem.lower().replace(" ", "_"))
    result[k] = newValue

print(result)

with open('angles.json', 'w') as f:
    json.dump(result, f)

{'frente': ['capot', 'farol_delantero_izquierdo', 'paragolpe_delantero', 'farol_delantero_derecho'], 'atras': ['panel_trasero', 'farol_trasero_derecho', 'farol_trasero_izquierdo', 'paragolpe_trasero', 'tapa_de_baul'], 'lado_cond': ['guardabarro_delantero_izquierdo', 'lateral_de_caja_izquierdo', 'puerta_trasera_izquierda', 'puerta_delantera_izquierda', 'zocalo_izquierdo', 'farol_trasero_izquierdo', 'guardabarro_trasero_izquierdo', 'farol_delantero_izquierdo'], 'lado_acomp': ['zocalo_derecho', 'puerta_delantera_derecha', 'farol_delantero_derecho', 'farol_trasero_derecho', 'lateral_de_caja_derecho', 'guardabarro_trasero_derecho', 'guardabarro_delantero_derecho', 'puerta_trasera_derecha'], 'frente_cond': ['guardabarro_delantero_izquierdo', 'paragolpe_delantero', 'lateral_de_caja_izquierdo', 'capot', 'puerta_delantera_izquierda', 'puerta_trasera_izquierda', 'zocalo_izquierdo', 'guardabarro_trasero_izquierdo', 'farol_delantero_izquierdo'], 'frente_acomp': ['zocalo_derecho', 'puerta_delantera